<a href="https://colab.research.google.com/github/AndrewSemenets/DSL/blob/main/job3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token : (class, value)    # токены
class : int
value : str
var : str                 # имя нетерминала
definition : list(rule)
rule : list(var | token)  # правая часть правила

```



Удаление бесполезных нетерминалов:
---

Непродуктивные: Заполняем список продуктивных нетерминалов, таких, у которых хотя бы в одном правиле все символы - терминалы или уже известные продуктивные нетерминалы. Все нетерминалы не попавшие в список - непродуктивные.

Недостижимые: Сначала в множестве достижимых нетерминалов только начальный нетерминал. Добавляем в множество достижимых нетерминалов все нетерминалы из правил нетерминалов, уже входящих в множество достижимых нетерминалов. Повторяем предыдущий шаг, если множество порождающих нетерминалов изменилось.

Далее удаляем из грамматики все непродуктивные нетерминалы и правила, в которых они упоминаются. Наконец удаляем из грамматики недостижимые нетерминалы.


In [188]:
def remove_external_symbs(grammar):

  toks = grammar['toks']
  vars = grammar['vars']
  

  #НЕПРОДУКТИВНЫЕ
  productive_symbs = set() #список продуктивных нетерминалов
  new_prod_found = True #флаг того, найден ли на предыдущей итерации новый продуктивный нетерминал

  #функция проверяющая, содержит ли правило только токены и продуктивные нетерминалы
  def check_rule(rule) -> bool:
    for rule_part in rule:
      if rule_part in toks or rule_part in productive_symbs:
        pass
      else:
        return False
    return True

  #ищем продуктивные нетерминалы
  while new_prod_found:
    new_prod_found = False
    for non_term, definition in vars.items():
      if non_term not in productive_symbs:
        for rule in definition:
          if check_rule(rule):
            new_prod_found = True
            productive_symbs.add(non_term)
            break
  
  #удаляем из грамматики все упоминания о непродуктивных нетерминалах (сами нетерминалы и правила с ними)
  new_vars = dict()
  for non_term, definition in vars.items():
    if non_term in productive_symbs:
      for rule in definition:
        if check_rule(rule):
          if non_term in new_vars.keys():
            new_vars[non_term].append(rule)
          else:
            new_vars[non_term] = list()
            new_vars[non_term].append(rule)

  grammar['vars'] = new_vars
  #------------------------------------------------------------

  #НЕДОСТИЖИМЫЕ
  reachable_symbs = set() #список достижимых нетерминалов
  reachable_symbs.add(grammar['hvar'])
  new_reach_found = True #флаг того, найден ли на предыдущей итерации новый достижимый нетерминал

  while new_reach_found:
    new_reach_found = False
    for non_term, definition in vars.items():
      if non_term in reachable_symbs:
        for rule in definition:
          for elem in rule:
            if elem not in toks and elem not in reachable_symbs:
              new_reach_found = True
              reachable_symbs.add(elem)
              
              
  #удаляем из грамматики все упоминания о недостижимых нетерминалах (сами нетерминалы и правила с ними)
  new_vars = dict()
  for non_term, definition in vars.items():
    if non_term in reachable_symbs:
      for rule in definition:
        if check_rule(rule):
          if non_term in new_vars.keys():
            new_vars[non_term].append(rule)
          else:
            new_vars[non_term] = list()
            new_vars[non_term].append(rule)
  
  grammar['vars'] = new_vars
  #------------------------------------------------------------

  return grammar

1) FIRST(X) - все терминалы, с которых могут начинаться всевозможные выводы из X.
---
 
 Для каждого правила нетерминала Х в множество FIRST(X) будет добавлятся следующее: 

 1) Если первый символ в правиле - терминал, добавляется этот терминал.

 2) Если первый символ в правиле - нетерминал A, то добавляется FIRST(A).



In [189]:
def first(X, rules, tokens):
  res_set = set()
  
  if len(rules) == 0:
    return res_set 
  
  for rule in rules[X]:
    if rule[0] in tokens:
      res_set.add(rule[0])
    else: 
      res_set = res_set.union(first(rule[0], rules, tokens)) 
      
  return res_set


2) FOLLOW(X) - всевозможные терминалы, которые встречаются после нетерминала Х во всех небесполезных правилах грамматики.
--
Чтобы найти follow для всех нетерминалов, нужно следовать алгоритму:

1)Если в правилах нетерминала A eсть правило  A -> aBb (где а и b - некоторые последовательности нетерминалов и терминалов), то все элементы first(b), за исключением символа конца ввода, добавить к follow(B).

2)Если в правилах нетерминала A есть правило A -> aB, то все элементы из follow(A) добавить к follow(B).

Повторять пункты 1 и 2, пока результирующий список follow изменяется.

In [190]:
def follow_for_every_nt(grammar):

  def first_for_seq(seq):
    result = set()
    for symbol in seq:
      if symbol in tokens:  
        result.add(symbol)
        return result
      else:
        result = set.union(result, dict_of_firsts[symbol])
        if endl_symb not in dict_of_firsts[symbol]:
          return result
        else:
          result.remove(endl_symb) 

    result.add(endl_symb)  
    return result
  #---------------------------------------

  nonterms = grammar['vars'].keys()
  tokens = grammar['toks']
  start_nt = grammar['hvar']

  res_dict = {}
  for nt in nonterms:
    res_dict[nt] = (set())

  dict_of_firsts = {}
  for nt in nonterms:
    dict_of_firsts[nt] = first(nt, grammar['vars'], tokens)
  
  endl_symb = (0, 'a')
  res_dict[start_nt].add(endl_symb)

  changed = True
  while changed:
    changed = False
    for nonterm, rules in grammar['vars'].items():
      for rule in rules:
        for index, symbol in enumerate(rule):
          if symbol in nonterms:
            end_seq_firsts = first_for_seq(rule[index+1:])
            prev_len = len(res_dict[symbol])

            if endl_symb in end_seq_firsts:
              end_seq_firsts.remove(endl_symb)
              res_dict[symbol] = res_dict[symbol].union(res_dict[nonterm]) #Если в правилах нетерминала A есть правило A -> aB, то все элементы из follow(A) добавить к follow(B).

            res_dict[symbol] = res_dict[symbol].union(end_seq_firsts) #Если в правилах нетерминала eсть правило  A -> aBb, то все элементы first(b),
                                                                      #за исключением символа конца ввода, добавить к follow(B).

            new_len = len(res_dict[symbol])
            if new_len != prev_len:
              changed = True
  return res_dict



  

Пример грамматики и вызов рабочих функций с ней
---

In [191]:
grammar1 = {
    'toks': set( [
        (0, 'a'), 
        (1, 'b'), 
        (1, 'c'), 
        (2, 'd')
    ] ),
    'vars': {
        'A' : [[(1, 'b')], 
               ['B', 'F'],
               [(0, 'a')],
               ['W', (1, 'b')]],
        'W' : [['F', (1, 'c')],
               [(0, 'a')]],
        'B' : [['D', (1, 'c')]],
        'C' : [['B', 'A', 'W']],
        'D' : [['B', (2, 'd')],
               ['C', (1, 'b')]],
        'F' : [[(2, 'd')],
               [(1, 'b'), 'B', 'W'],]
    },
    'hvar': 'F'
}


#print("Our grammar without the external non-terminals: ", remove_external_symbs(grammar))
grammar1 = remove_external_symbs(grammar1)
print(grammar1)

nt = 'A'
print("first(" + nt + "): ", first(nt, grammar1['vars'], grammar1['toks']))

print("follow: ", follow_for_every_nt(grammar1))


{'toks': {(1, 'b'), (2, 'd'), (0, 'a'), (1, 'c')}, 'vars': {'A': [[(1, 'b')], [(0, 'a')], ['W', (1, 'b')]], 'W': [['F', (1, 'c')], [(0, 'a')]], 'F': [[(2, 'd')]]}, 'hvar': 'F'}
first(A):  {(1, 'b'), (2, 'd'), (0, 'a')}
follow:  {'A': set(), 'W': {(1, 'b')}, 'F': {(0, 'a'), (1, 'c')}}
